In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Generate some fake data.
x_concs = np.arange(0, 100, 1).reshape(-1, 1)
log_xconcs = np.log(x_concs)
ic50_true = np.array([42, 13, 88])
ic50_true = ic50_true.reshape(-1, ic50_true.shape[0])
beta_true = 1
slope_true = 1
intercept_true = 150
# y_true = slope_true * x_concs + intercept_true

y_true = beta_true / (1 + np.exp(x_concs - ic50_true))

y_noisy = y_true + np.random.normal(0, 0.15, size=y_true.shape)  # homoskedastic error
y_noisy

In [ ]:
plt.scatter(x_concs, y_noisy[:, 0])
plt.scatter(x_concs, y_noisy[:, 1])
plt.scatter(x_concs, y_noisy[:, 2])

In [ ]:
concentrations = np.concatenate([x_concs] * ic50_true.shape[1])
concentrations

In [ ]:
concentrations.shape

In [ ]:
y_noisy.flatten(order='F')

In [ ]:
x_concs.shape[0]

In [ ]:
data = pd.DataFrame()
data['concentrations'] = concentrations.reshape(concentrations.shape[0],)
data['measurements'] = y_noisy.flatten(order='F')

drugs = []
for i in range(ic50_true.shape[1]):
    drugs.extend([i] * x_concs.shape[0])
data['drug'] = drugs

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

le = LabelEncoder()
data['idxs'] = le.fit_transform(data['drug'])

# Normalize data['measurements'] to 0-1
# mms = MinMaxScaler()
# data['measurements'] = mms.fit_transform(data['measurements'].values.reshape(-1, 1))

In [ ]:
data

In [ ]:
with pm.Model() as model:
    beta = pm.Normal('beta', mu=0, sd=100**2, shape=len(set(data['idxs'])))    
    noise = pm.HalfCauchy('noise', beta=100**2, shape=len(set(data['idxs'])))
    ic50 = pm.Normal('IC50', sd=100**2, shape=len(set(data['idxs'])))
    measurements = beta[data['idxs']] / (1 + np.exp(data['concentrations'].values - ic50[data['idxs']]))

    
    y_like = pm.Normal('y_like', mu=measurements, sd=noise[data['idxs']], observed=data['measurements'])

In [ ]:
with model:
    trace = pm.sample(draws=10000, step=pm.Metropolis(), start=pm.find_MAP())

In [ ]:
pm.traceplot(trace)

In [ ]:
pm.summary(trace)